# Hashlookup Analysis Workshop

This workshop introduces Hashlookup, a powerful file hash intelligence service for malware analysis and threat hunting. Hashlookup provides comprehensive information about known files through their cryptographic hashes.

## What is Hashlookup?

Hashlookup is a public API service that provides file intelligence based on cryptographic hashes. It offers:
- **File Intelligence**: Detailed information about known files from various sources
- **Hash Correlation**: Cross-referencing between MD5, SHA-1, and SHA-256 hashes
- **Metadata Enrichment**: File names, sizes, types, and associated information
- **Threat Classification**: Known good/bad file classifications from multiple sources
- **Historical Data**: Timeline information about file observations

## Key Features

### Hashlookup Capabilities:
- **Multi-Hash Support**: Query using MD5, SHA-1, or SHA-256 hashes
- **Bulk Operations**: Process multiple hashes in a single API call
- **File Metadata**: Retrieve comprehensive file information including names and sizes
- **Source Attribution**: Information about where files have been observed
- **Known Good/Bad Classification**: Threat intelligence about file reputation
- **API Integration**: RESTful API for programmatic access and automation

## Data Sources

Hashlookup aggregates data from multiple sources:
- **NSRL (National Software Reference Library)**: Known good software files
- **Threat Intelligence Feeds**: Malware and threat indicators
- **Sandbox Analysis**: Dynamic analysis results from security platforms
- **Community Submissions**: Crowdsourced file intelligence
- **Vendor Databases**: Commercial security product databases

## Documentation

- **Main Service**: https://circl.lu/services/hashlookup
- **API Documentation**: https://hashlookup.circl.lu/

### Use Cases:
- **Malware Analysis**: Identify known malicious files and their characteristics
- **Incident Response**: Quickly assess file reputation during investigations
- **Threat Hunting**: Search for indicators of compromise across file systems
- **Digital Forensics**: Validate file integrity and identify unknown files
- **Security Operations**: Automate file reputation checks in security workflows

## Learning Objectives

By the end of this workshop, you will be able to:
1. Query individual file hashes for detailed intelligence
2. Perform bulk hash lookups for efficient analysis
3. Integrate Hashlookup with local file system analysis
4. Correlate MISP threat intelligence with Hashlookup data
5. Implement automated file reputation workflows
6. Analyze and interpret file intelligence results

## Exercises

In [ ]:
import requests
import json

# Configure HTTP session for API communication
print("Configuring API session for Hashlookup...")

# Hashlookup session configuration
hashlookup = requests.Session()

print("API session configured successfully!")
print("Ready to query Hashlookup service.")

### Exercise 1.0: Query Individual File Hash Intelligence

**Objective:** Learn how to retrieve comprehensive file intelligence for a specific hash value.

**Understanding Hash-Based File Intelligence:**
Cryptographic hashes serve as unique fingerprints for files, enabling:
- **File Identification**: Uniquely identify files regardless of name or location
- **Integrity Verification**: Confirm files haven't been modified or corrupted
- **Threat Detection**: Identify known malicious files through their hashes
- **Duplicate Detection**: Find identical files across different systems or networks

**Hash Types Supported:**
- **MD5**: 32-character hexadecimal string (legacy but widely used)
- **SHA-1**: 40-character hexadecimal string (more secure than MD5)
- **SHA-256**: 64-character hexadecimal string (current security standard)

**About This Hash:**
The MD5 hash `aaade8e2a921e9ac40178a263ebb67e3` represents a specific file. Through Hashlookup, we can discover:
- What file this hash represents
- Where it has been observed
- Whether it's classified as malicious or benign
- Associated metadata and file information

**API Endpoint:** `https://hashlookup.circl.lu/lookup/md5/[hash]`

**Security Applications:**
This information helps security analysts:
- Identify unknown files during incident response
- Validate file integrity during forensic analysis
- Assess threat level of suspicious files
- Build comprehensive file intelligence profiles

In [ ]:
md5 = "aaade8e2a921e9ac40178a263ebb67e3"
response = hashlookup.get(f"https://hashlookup.circl.lu/lookup/md5/{md5}")
if response.status_code == 200:
    data = response.json()
    print("Hashlookup Result:", json.dumps(data, indent=4))
else:
    print("Error querying Hashlookup:", response.status_code)

### Exercise 1.1: Bulk Hash Intelligence Queries

**Objective:** Demonstrate efficient batch processing of multiple file hashes for large-scale analysis.

**Understanding Bulk Operations:**
When analyzing multiple files or conducting threat hunting across large datasets, individual hash queries become inefficient. Bulk operations provide:
- **Efficiency**: Process hundreds of hashes in a single API call
- **Rate Limit Compliance**: Avoid overwhelming the API with individual requests
- **Comprehensive Analysis**: Get complete intelligence for entire file sets
- **Workflow Integration**: Enable automated processing of hash lists

**Bulk Query Benefits:**
- **Performance**: Significantly faster than individual queries
- **Resource Optimization**: Reduced network overhead and API calls
- **Batch Processing**: Ideal for automated security workflows
- **Scale**: Handle large incident response or forensic investigations

**API Endpoint:** `https://hashlookup.circl.lu/bulk/md5`

* **Hint**: You can also do bulk `SHA1` queries using `https://hashlookup.circl.lu/bulk/sha1`.

In [ ]:
hashes = [
    "aaade8e2a921e9ac40178a263ebb67e3",
    "30caea6cf2c12bbb1b626b04ab77d7cb",
    "d04249be1ba985d1f94aea79deab52de",
]

response = hashlookup.post(
    f"https://hashlookup.circl.lu/bulk/md5", json={"hashes": hashes}
)
if response.status_code == 200:
    data = response.json()
    print("Hashlookup Result:", json.dumps(data, indent=4))
else:
    print("Error querying Hashlookup:", response.status_code)

### Exercise 1.2: File System Analysis and Unknown File Detection

**Objective:** Integrate Hashlookup with local file system analysis to identify unknown or potentially suspicious files.

**Understanding File System Intelligence:**
This exercise demonstrates real-world application of hash-based file intelligence by:
- **System Auditing**: Analyze files present on local systems
- **Baseline Establishment**: Identify which files are known and documented
- **Anomaly Detection**: Find files that aren't in public databases
- **Security Assessment**: Evaluate the security posture of file systems

**Why Analyze System Directories:**
System directories like `/usr/sbin` contain critical system utilities and administrative tools:
- **High-Value Targets**: Attackers often target system directories for persistence
- **Integrity Verification**: Ensure system files haven't been tampered with
- **Baseline Security**: Establish known-good file inventories
- **Change Detection**: Identify unauthorized modifications or additions

In [ ]:
import os
import hashlib

directory = "/usr/sbin"
hashes = []
hashes_filenames = {}
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if os.path.isfile(filepath):
        with open(filepath, "rb") as f:
            file_data = f.read()
            file_hash = hashlib.md5(file_data).hexdigest()
            hashes.append(file_hash)
            hashes_filenames[file_hash] = filename
print(f"Computed {len(hashes)} hashes from files in {directory}")

response = hashlookup.post(
    f"https://hashlookup.circl.lu/bulk/md5", json={"hashes": hashes}
)
if response.status_code == 200:
    data = response.json()
else:
    print("Error querying Hashlookup:", response.status_code)

print("Files with not found hashes:")
not_found_count = 0
for file_hash in hashes:
    found = False
    for result in data:
        if result["MD5"].upper() == file_hash.upper():
            found = True
            break
    if not found:
        print(f"- {hashes_filenames.get(file_hash, 'Unknown file')}: {file_hash}")
        not_found_count += 1
print(f"Total not found hashes: {not_found_count}")

### Exercise 1.3: MISP and Hashlookup Intelligence Correlation

**Objective:** Demonstrate integration between MISP threat intelligence platform and Hashlookup file intelligence service.

**Understanding Intelligence Correlation:**
This exercise shows how to combine different threat intelligence sources:
- **MISP Integration**: Extract file hashes from threat intelligence events
- **Hash Enrichment**: Enhance MISP indicators with Hashlookup intelligence
- **Cross-Platform Analysis**: Correlate data from multiple intelligence sources
- **Comprehensive Profiling**: Build complete threat intelligence pictures

**MISP as Hash Source:**
MISP contains extensive collections of file hashes from:
- **Malware Analysis**: Hashes of analyzed malicious files
- **Incident Reports**: IOCs from security incidents and breaches
- **Threat Intelligence Feeds**: Commercial and open source threat data
- **Community Sharing**: Collaborative threat intelligence from security teams

**Intelligence Enhancement Process:**
1. **Hash Extraction**: Retrieve MD5 hashes from MISP attribute database
2. **Bulk Analysis**: Submit MISP hashes to Hashlookup for enrichment
3. **Data Correlation**: Match Hashlookup results with MISP context
4. **Intelligence Fusion**: Combine threat context with file intelligence

**Use Cases:**
- **Threat Hunting**: Search for known malicious files across enterprise systems
- **IOC Validation**: Verify threat indicators before deploying to security tools
- **Incident Response**: Quickly assess file reputation during investigations
- **Intelligence Analysis**: Build comprehensive threat actor profiles
### Exercise 1.3: MISP and Hashlookup Intelligence Correlation

**Objective:** Demonstrate integration between MISP threat intelligence platform and Hashlookup file intelligence service.

**Understanding Intelligence Correlation:**
This exercise shows how to combine different threat intelligence sources:
- **MISP Integration**: Extract file hashes from threat intelligence events
- **Hash Enrichment**: Enhance MISP indicators with Hashlookup intelligence
- **Cross-Platform Analysis**: Correlate data from multiple intelligence sources
- **Comprehensive Profiling**: Build complete threat intelligence pictures

**MISP as Hash Source:**
MISP contains extensive collections of file hashes from:
- **Malware Analysis**: Hashes of analyzed malicious files
- **Incident Reports**: IOCs from security incidents and breaches
- **Threat Intelligence Feeds**: Commercial and open source threat data
- **Community Sharing**: Collaborative threat intelligence from security teams

**Intelligence Enhancement Process:**
1. **Hash Extraction**: Retrieve MD5 hashes from MISP attribute database
2. **Bulk Analysis**: Submit MISP hashes to Hashlookup for enrichment
3. **Data Correlation**: Match Hashlookup results with MISP context
4. **Intelligence Fusion**: Combine threat context with file intelligence

**Use Cases:**
- **Threat Hunting**: Search for known malicious files across enterprise systems
- **IOC Validation**: Verify threat indicators before deploying to security tools
- **Incident Response**: Quickly assess file reputation during investigations
- **Intelligence Analysis**: Build comprehensive threat actor profiles

**Analysis Questions:**
- How many MISP hashes were successfully enriched vs not found in Hashlookup?
- Are there hashes with conflicting reputations between sources? How should they be prioritized?
- Which additional fields from Hashlookup (filename, size, sources) helped confirm or refute MISP context?
- What confidence thresholds will you use to automate tagging or blocking actions?
- Which parts of this workflow can be safely automated, and which require analyst review?

In [ ]:
import pymisp
import urllib3
import getpass

MISP_BASEURL = "https://training.misp-community.org"
MISP_API_KEY = getpass.getpass("Enter your MISP AuthKey:")

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

misp = pymisp.PyMISP(MISP_BASEURL, MISP_API_KEY, ssl=False)  # Disable SSL verification
print(f"Connected to MISP {misp.root_url} running version: {misp.version['version']}")

In [ ]:
attributes = misp.search(
    "attributes",
    type_attribute=["md5"],
    pythonify=True,
)
hashes = [attr.value for attr in attributes]
print(f"Retrieved {len(hashes)} MD5 hashes from MISP.")

response = hashlookup.post(
    f"https://hashlookup.circl.lu/bulk/md5", json={"hashes": hashes}
)
if response.status_code == 200:
    print ("Hashlookup Result:", json.dumps(response.json(), indent=4))
else:
    print("Error querying Hashlookup:", response.status_code)

### Homework - Use other third-party services to investigate filehashes
- Use the VirusTotal web UI or API to collect a small set of MD5 hashes (export a CSV or copy a few sample hashes).
- Put those MD5 values into the notebook (e.g., assign them to the existing `hashes` list or save to a file and load them).
- Reuse the already-configured `hashlookup` session and run the notebook cell that posts to the `/bulk/md5` endpoint to enrich those hashes.
- After enrichment, check:
    - how many hashes were found vs not found,
    - filenames/sizes/sources returned,
    - any reputation or vendor classifications indicating maliciousness.

In [ ]:

# Your code here...
